In [1]:
import pyautogui
import time
from unidecode import unidecode
import pandas as pd
import numpy as np
import os
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [2]:
screenWidth, screenHeight = pyautogui.size()

originalScreenWidth = 1920
originalScreenHeight = 1080

In [3]:
time.sleep(3)
pyautogui.position()

Point(x=1281, y=598)

In [4]:
time.sleep(3)
im = pyautogui.screenshot(region=(360,620,110,30))
im#.save('wait_for_this (1).png')

In [5]:
#pytesseract.image_to_string(im).strip()

In [6]:
def read_screen(x_start, y_start, x_end, y_end):
    global screenWidth, screenHeight, originalScreenWidth, originalScreenHeight
    region = (
        int(x_start * screenWidth / originalScreenWidth),
        int(y_start * screenHeight / originalScreenHeight),
        int(x_end * screenWidth / originalScreenWidth),
        int(y_end * screenHeight / originalScreenHeight)
    )
    im = pyautogui.screenshot(region=(x_start, y_start, x_end, y_end))
    return pytesseract.image_to_string(im).strip()

In [7]:
def click(x, y):
    global screenWidth, screenHeight, originalScreenWidth, originalScreenHeight
    pyautogui.click(
        int(x * screenWidth / originalScreenWidth),
        int(y * screenHeight / originalScreenHeight)
    )

In [8]:
def lookup_company(empresa):

    global log

    # empty the search bar
    click(180, 85)
    click(180, 85)
    pyautogui.press('backspace')

    # type the company name
    pyautogui.typewrite(empresa)
    time.sleep(2)

    # wait for the name to show and select it
    start_time = time.time()
    while read_screen(50,115,275,25) == 'Nao ha resultados para a busca.':
        if time.time() - start_time > 5:
            log.write(f"{empresa} - 'Not found'\n")
            return False
        
    pyautogui.press('down')
    pyautogui.press('enter')

    # while Dados Cadastrais not available click it
    while not pyautogui.locateOnScreen('wait_for_this (Dados Cadastrais).png'):
        click(180, 300)

    return True

In [9]:
#time.sleep(3)
#lookup_company('Pomifrutas S.A.')

In [10]:
def select_fields():
    
    start_time = time.time()
    while (time.time() - start_time) < 2:  # 2 seconds timeout
        location = pyautogui.locateOnScreen('wait_for_this (Padrao).png', confidence=0.9)
        if location is not None:
            time.sleep(0.3)
            click(580, 165)
            time.sleep(0.1)
            click(640, 320)
            time.sleep(0.1)
            click(910, 500)
            time.sleep(0.1)
            click(910, 580)
            time.sleep(0.1)
            click(1165, 570)
            break
    else:
        return

In [11]:
def reset_program():

    os.system('taskkill /IM "ValorPRO.exe" /F')
    pyautogui.hotkey('win', 'd')
    time.sleep(0.5)
    click(60, 380)
    click(60, 380)

    start_time = time.time()
    while not pyautogui.locateOnScreen('wait_for_this (Valor PRO).png', confidence=0.9):
        if time.time() - start_time > 60:
            return
        
    click(1015, 365)

    click(390, 420)

    time.sleep(10)
    pyautogui.click(pyautogui.center(pyautogui.locateOnScreen('wait_for_this (Full Screen).png', confidence=0.9)))

In [12]:
def download(path):

    global log, empresa

    downloaded_dates = [x.split('__')[-1].split('.')[0] for x in os.listdir(path)]

    # current date being downloaded
    date = read_screen(335,160,110,20).replace('/', '_')

    if date not in downloaded_dates:
        
        # wait for VALORPRO to finish loading the data
        start_time = time.time()
        while not read_screen(330,240,150,25) == 'Administracao':
            if time.time() - start_time > 2:
                log.write(f"{empresa} - {date} - 'Administracao field not found'\n")
                return

        # position of the 'printer' button
        time.sleep(0.3)
        click(490, 90)

        # wait for the print menu to appear
        start_time = time.time()
        while (time.time() - start_time) < 5:
            location1 = pyautogui.locateOnScreen('wait_for_this (1).png', confidence=0.9)
            location2 = pyautogui.locateOnScreen('wait_for_this (Print).png', confidence=0.9)
            if location1 is not None:
                time.sleep(0.3)
                pyautogui.click(pyautogui.center(location1))
                found = True
                break
            elif location2 is not None:
                time.sleep(0.3)
                pyautogui.click(pyautogui.center(location2))
                found = True
                break
            found = False

        # if the print menu didn't appear, reset the program
        if not found:
            reset_program()
            pass
            return

        # wait for the file explorer menu to appear
        start_time = time.time()
        while (time.time() - start_time) < 5:
            location = pyautogui.locateOnScreen('wait_for_this (Save Print).png', confidence=0.9)
            if location is not None:
                found = True
                break
            found = False
        
        # if the file explorer menu didn't appear, reset the program
        if not found:
            reset_program()
            pass
            return

        # position of the 'path'
        time.sleep(0.3)
        click(500, 80)

        # go to path
        time.sleep(0.3)
        pyautogui.typewrite(path)
        pyautogui.press('enter')

        # name of the file
        time.sleep(0.3)
        click(240, 550)
        pyautogui.typewrite(f"{path.split('/')[-2]}__{date}")

        time.sleep(1)
        # save
        click(700, 670)
        time.sleep(3)

In [13]:
# empty log file
open('log.txt', 'w').close()

# open new log file
log = open('log.txt', 'w')

# time for the user to put VALORPRO in full screen 
time.sleep(5)

# companies csv
empresas_df = pd.read_csv('valorPRO_empresas.csv', index_col=0)

# missing companies list
empresas = empresas_df.iloc[np.where(np.logical_and(empresas_df['collected'] != 'X', empresas_df['collected'] != 'unavailable'))]
empresas = empresas.index.to_list()
empresas = [unidecode(x) for x in empresas]

# channel directory
full_path = os.path.abspath('automation.ipynb').split('automation.ipynb')[0].replace('\\', '/') + 'empresas/'

# loop through the companies and available dates
for empresa in empresas:

    # create the company directory
    path = f"{full_path}{empresa.strip().strip('.').replace('/', '_')}/"
    if not os.path.exists(path):
        os.mkdir(path.strip())

    # lookup the company and select 'Dados Cadastrais'
    if not lookup_company(empresa.strip()):
        # mark the company as unavailable
        empresas_df.loc[empresa, 'collected'] = 'unavailable'
        empresas_df.to_csv('valorPRO_empresas.csv')
        continue
    
    # select the fields
    select_fields()

    # download the first date
    download(path)

    # loop the available dates
    time.sleep(1)
    last_date = True
    start_time = time.time()
    while last_date:
        
        time.sleep(1)
        click(400, 160)
        time.sleep(0.5)
        # check if the date is the last one
        last_date = read_screen(335, 525, 110, 20).replace('/', '').isdigit()
        click(380, 245)
        download(path)

    for i in range(2, 11):
        time.sleep(1)
        click(400, 160)
        time.sleep(0.5)
        click(380, (210 + 33*i))
        download(path)

    # mark the company as collected
    empresas_df.loc[empresa, 'collected'] = 'X'
    empresas_df.to_csv('valorPRO_empresas.csv')

OSError: screen grab failed

In [ ]:
## IN THE FUTURE TRANSFORM PDFS TO CSVs
# Remember: there may be some companies with unmatching data